# Webscraping - appliedai.com

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
import time
import re

%matplotlib inline
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import interactive

In [ ]:
# https://duo.com/decipher/driving-headless-chrome-with-python
# Install Chrome Canary
# Download latest Chromedriver (to Downloads)
# !mkdir ../driver_headless

In [ ]:
# !cp ~/Downloads/chromedriver ~/frank/lauthom_intel_design_studio/_driver_headless/

In [ ]:
!pwd
!find .. | grep headless

## Headless Selenium

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait 
# from selenium.webdriver.support import expected_conditions as EC 
# from selenium.common.exceptions import TimeoutException

In [ ]:
chrome_options = Options()  
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")
chrome_options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'

In [ ]:
url = 'https://appliedai.com/use-cases'
keyword = '/1'
url += keyword

In [ ]:
driver_path = '../_driver_headless/chromedriver'
driver = webdriver.Chrome(executable_path=os.path.abspath(driver_path), chrome_options=chrome_options)  
driver.get(url)

In [ ]:
print(driver.name, driver.title, driver.current_url, driver.get_cookies)

## Scrape with Xpath
 - NOTE: to go to the start again, rerun **`driver.get(url)`**

In [ ]:
# rerun driver
xpath = '//*[@id="__next"]/div/section/div[1]/div/div[3]/div[2]/div/div/div/table/*'
SCROLL_PAUSE_TIME = 5
text = []
print('loading use cases: ')
for i in range(1, 15):
    try:
        for tag in driver.find_elements_by_xpath(xpath):
#             print(tag.text)
            text.append('{}\n'.format(tag.text))
        # Goto next page / find child-name-tag and then parent-tag = '..'
        child = driver.find_element_by_xpath('//*[@class="right chevron icon"]')
        child.find_element_by_xpath('..').click()  # click parent-tag
#         print(driver.current_url)
        time.sleep(SCROLL_PAUSE_TIME)
    except:
        continue
    finally:
        print('{}{}'.format('\b'*len(str(len(text))), len(text)), end='')

In [ ]:
text #[0].split('\n')

## Convert to DataFrame
 - sanity check for columns and content

In [ ]:
df = pd.DataFrame([x.split('\n') for x in text])
df.info()
df

In [ ]:
df.columns = 'use_case function description bizz_metrics 4 5 6 7 8 9 10'.split(' ')
df = df.fillna('')
df.replace(r'Use Case ', r'', regex=True, inplace=True)
df.replace(r'Function ', r'', regex=True, inplace=True)
df.replace(r'Description ', r'', regex=True, inplace=True)
df.replace(r'References.*', r'', regex=True, inplace=True)
df.replace(r'See Details.*', r'', regex=True, inplace=True)
df['business_metrics'] = df['4'] + ', ' + df['5'] + ', ' + df['6'] + ', ' + df['7'] + ', ' + df['8'] + ', ' + df['9'] + ', ' + df['10']
df.replace(r', ,', r'', regex=True, inplace=True)
df.replace(r' ,', r'', regex=True, inplace=True)
del df['bizz_metrics']
del [df['4'], df['5'], df['6'], df['7'], df['8'], df['9'], df['10']]

In [ ]:
writer = pd.ExcelWriter('../_data/applied_AI.xlsx')

# index startrow/startcol N means value is inserted at N+1
df.to_excel(writer, 'AI')
writer.save()

In [ ]:
pd.read_excel('../_data/applied_AI.xlsx')

In [ ]:
# https://pandas-docs.github.io/pandas-docs-travis/options.html
pd.set_option('display.max_colwidth', 1000)
pd.reset_option('colheader_justify')
pd.set_option('colheader_justify', 'left')

items = ['All'] + sorted(df['use_case'].unique().tolist())
w = widgets.Select(options=items)

def view(x=''):
    if x=='All': return display(HTML(df.to_html())) # display(df)
    return display(HTML(df.loc[df['use_case']==x, ['description']].to_html()))
 
interactive(view, x=widgets.Select(options=items))

In [ ]:
# display(HTML(df.to_html()))
# display(df)